In [2]:
# === colab configuration ===
# p.s. when training on colab, weights are saved on Drive (directory DSCAS/weights).
# p.p.s. skip this cell if running demo file locally!

! pip install torch-summary
! pip install ray
from google.colab import drive
import sys

# setting paths
repository_path = '/content/deep-seasonal-color-analysis-system/'
dataset_path = repository_path + 'headsegmentation_dataset_ccncsa/'
dataset_path_drive = '/content/drive/MyDrive/DSCAS/headsegmentation_dataset_ccncsa/'
weights_path = repository_path + 'models/weights/'
weights_path_drive = '/content/drive/MyDrive/DSCAS/weights/'
sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/mrcmich/deep-seasonal-color-analysis-system.git
! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'fastscnn_hyperparameters_tuning__refactoring'
! git checkout $branch
! git pull origin $branch

executing_on_colab = True

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/deep-seasonal-color-analysis-system
Updating files: 100% (121/121), done.
Branch 'fastscnn_hyperparameters_tuning__refactoring' set up to track remote branch 'fastscnn_hyperparameters_tuning__refactoring' from 'origin'.
Switched to a new branch 'fastscnn_hyperparameters_tuning__refactoring'
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), 772 bytes | 772.00 KiB/s, done.
From https://github.com/mrcmich/deep-seasonal-color-analysis-system
 * branch            fastscnn_hyperparameters_tunin

In [3]:
import torch
from torch import nn
import torchvision.transforms as T
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from models import dataset, training_and_testing
from models.local.FastSCNN.models import fast_scnn
from metrics_and_losses import metrics
from utils import segmentation_labels, utils
import matplotlib.pyplot as plt
from palette_classification import color_processing
import torchsummary
from models import config


dataset_path = config.ROOT_DIR + 'headsegmentation_dataset_ccncsa/'

# defining transforms
tH, tW = 256, 256 # image sizes must be multiple of 16
transform = T.Compose([T.CenterCrop((tH, tW))])

# fetching dataset
n_classes = len(segmentation_labels.labels)
img_paths, label_paths = dataset.get_paths(dataset_path, file_name=config.DATASET_INDEX_NAME)
X_train, X_test, Y_train, Y_test = train_test_split(img_paths, label_paths, test_size=0.20, random_state=99, shuffle=True)
train_dataset = dataset.CcncsaDataset(X_train, Y_train, transform)

# data loader
image_loader = DataLoader(train_dataset, 
                          batch_size=32,
                          drop_last=False)

psum = torch.zeros((3), dtype=torch.float32)
psum_sq = torch.zeros((3), dtype=torch.float32)

for image, _ in image_loader:
    psum += image.sum(dim=[0, 2, 3])
    psum_sq += (image ** 2).sum(dim=[0, 2, 3])

count = len(train_dataset) * tH * tW

mean = psum / count
var  = (psum_sq / count) - (mean ** 2)
std  = torch.sqrt(var)
print(f'mean: {mean}')
print(f'var: {var}')
print(f'std: {std}')

mean: tensor([0.3954, 0.3269, 0.2831])
var: tensor([0.0632, 0.0555, 0.0533])
std: tensor([0.2513, 0.2356, 0.2309])
